In [2]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
import os


In [3]:
load_dotenv(dotenv_path = "password.env") 

# MONGO_URI example: MONGO_URI=mongodb://localhost:27017/

mongo_uri = os.getenv("mongo_uri")
mongo_database = os.getenv("mongo_database")

client = MongoClient(mongo_uri)

db = client[mongo_database]

In [121]:
# Loading in csv data
patrons_data = pd.read_csv('patrons.csv')
inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
inventory['isbn'] = inventory['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')


# Converting to dictionaries
patrons_dict = patrons_data.to_dict('records')
loans_dict = loans.to_dict('records')
books_dict = books.to_dict('records')
inventory_dict = inventory.to_dict('records')
authors_dict = authors.to_dict('records')

# Adding authors as a subset of books
for item in books_dict:
    item['author'] = {}
for item in authors_dict:
    author_id = item['author_id']
    for book in books_dict:
        if book['author_id'] == author_id:
            book['author'] = item

# Merging books' meta data with inventoried books
for item in books_dict:
    isbn = item['isbn']
    for book in inventory_dict:
        if book['isbn'] == isbn:
            book['title'] = item['title']
            book['pub_year'] = item['pub_year']
            book['genre'] = item['genre']
            book['overdue_fee_rate'] = item['overdue_fee_rate']
            book['author'] = item['author']

# Adding inventories books as a subset of loans
for item in loans_dict:
    item['book'] = {}
for item in inventory_dict:
    book_id = item['book_id']
    for loan in loans_dict:
        if loan['book_ID'] == book_id:
            loan['book'] = item
    item.pop('book_id')

# Inserting total overdue fees per patron
for patron in patrons_dict:
    patron['loan_history'] = []
    patron['total_fees_owed'] = 0
    for item in loans_dict:
        if pd.isna(days):
            days = 0
        rate = item['book']['overdue_fee_rate']
        fee = rate * days
        if item['patron_ID'] == patron['patron_id']:
            patron['total_fees_owed'] += fee

# Adding loans as a subset of patrons
for item in loans_dict:
    patron_id = item['patron_ID']
    for patron in patrons_dict:
        if patron['patron_id'] == patron_id:
            patron['loan_history'].append(item)
            item.pop("loan_ID")
            item.pop("book_ID")
            item.pop('patron_ID')
            break


# Getting rid of `patron_id` variable in patrons
for item in patrons_dict:
    item.pop('patron_id', None)

patrons_dict[-4]


{'first_name': 'Chloe',
 'last_name': 'Robinson',
 'address': '305 Redwood Circle',
 'phone_number': '101-010-1010',
 'age': 29,
 'loan_history': [{'loan_date': '2024-12-27',
   'due_date': '2025-01-26',
   'return_date': '2025-01-29',
   'days_overdue': 3.0,
   'book': {'isbn': '9780374528379',
    'condition': 'Fair',
    'title': 'Interpreter of Maladies',
    'pub_year': 1999,
    'genre': 'Short Stories',
    'overdue_fee_rate': 0.25,
    'author': {'author_id': 19,
     'first_name': 'Jhumpa',
     'last_name': 'Lahiri',
     'nationality': 'Indian-American',
     'language': 'English',
     'birth_year': '1967',
     'biography': "Jhumpa Lahiri won the Pulitzer Prize for 'Interpreter of Maladies'."}}},
  {'loan_date': '2025-05-22',
   'due_date': '2025-06-21',
   'return_date': nan,
   'days_overdue': nan,
   'book': {'isbn': '9780812981602',
    'condition': 'Fair',
    'title': 'The Vegetarian',
    'pub_year': 2007,
    'genre': 'Literary Fiction',
    'overdue_fee_rate': 0.5

In [123]:
# PATRONS COLLECTION

# Adding patron objects to MongoDB
patrons = db.patrons
patrons.insert_many(patrons_dict)

# Delete current objects to repopulate if necessary
# patrons.delete_many({})

InsertManyResult([ObjectId('683737a160f44cae1186ad6b'), ObjectId('683737a160f44cae1186ad6c'), ObjectId('683737a160f44cae1186ad6d'), ObjectId('683737a160f44cae1186ad6e'), ObjectId('683737a160f44cae1186ad6f'), ObjectId('683737a160f44cae1186ad70'), ObjectId('683737a160f44cae1186ad71'), ObjectId('683737a160f44cae1186ad72'), ObjectId('683737a160f44cae1186ad73'), ObjectId('683737a160f44cae1186ad74'), ObjectId('683737a160f44cae1186ad75'), ObjectId('683737a160f44cae1186ad76'), ObjectId('683737a160f44cae1186ad77'), ObjectId('683737a160f44cae1186ad78'), ObjectId('683737a160f44cae1186ad79'), ObjectId('683737a160f44cae1186ad7a'), ObjectId('683737a160f44cae1186ad7b'), ObjectId('683737a160f44cae1186ad7c'), ObjectId('683737a160f44cae1186ad7d'), ObjectId('683737a160f44cae1186ad7e'), ObjectId('683737a160f44cae1186ad7f'), ObjectId('683737a160f44cae1186ad80'), ObjectId('683737a160f44cae1186ad81'), ObjectId('683737a160f44cae1186ad82'), ObjectId('683737a160f44cae1186ad83'), ObjectId('683737a160f44cae1186ad

In [84]:
# BOOKS COLLECTION

# Adding book objects to MongoDB
books = db.books
books.insert_many(inventory_dict)

# Delete current objects to repopulate if necessary
# books.delete_many({})

InsertManyResult([ObjectId('6837320d60f44cae1186aca8'), ObjectId('6837320d60f44cae1186aca9'), ObjectId('6837320d60f44cae1186acaa'), ObjectId('6837320d60f44cae1186acab'), ObjectId('6837320d60f44cae1186acac'), ObjectId('6837320d60f44cae1186acad'), ObjectId('6837320d60f44cae1186acae'), ObjectId('6837320d60f44cae1186acaf'), ObjectId('6837320d60f44cae1186acb0'), ObjectId('6837320d60f44cae1186acb1'), ObjectId('6837320d60f44cae1186acb2'), ObjectId('6837320d60f44cae1186acb3'), ObjectId('6837320d60f44cae1186acb4'), ObjectId('6837320d60f44cae1186acb5'), ObjectId('6837320d60f44cae1186acb6'), ObjectId('6837320d60f44cae1186acb7'), ObjectId('6837320d60f44cae1186acb8'), ObjectId('6837320d60f44cae1186acb9'), ObjectId('6837320d60f44cae1186acba'), ObjectId('6837320d60f44cae1186acbb'), ObjectId('6837320d60f44cae1186acbc'), ObjectId('6837320d60f44cae1186acbd'), ObjectId('6837320d60f44cae1186acbe'), ObjectId('6837320d60f44cae1186acbf'), ObjectId('6837320d60f44cae1186acc0'), ObjectId('6837320d60f44cae1186ac